# 5293_final project codes

### by Jiapei Li(jl6277), Junkai Ding(jd3868), Taimin Chen(tc3233)

In [1]:
import pandas as pd
import numpy as np

In [2]:
# discrete Fourier transform
from scipy.fft import fft

# [0] create dataframe and Features (Only consider local predictor)

### <span style="color:blue;">Since data are fully fictional, to train a global predictor or a local predictor are without any practical difference; In addition, with the same framework one can easily enough train a global predictor; Therefore we only consider training a local predictor here

### [0.1]for the expeirimental period, we create a (500 users)*(21 days) dataframe for key metrics X

In [3]:
# We need to create two dataset, each one is of 21 days duration and of 500 datapoint,where 250 as treatment group
np.random.seed(46)
# df1 is the pre-experimental period and experimental period, to get local predictor
df1 = pd.DataFrame(columns=['X'+str(i+1) for i in range(21)], index=range(500))
df1.iloc[:250, :] = np.random.poisson(5, size=(250, 21)) + np.random.randint(0, 4, size=(250, 21))
df1.iloc[250:, :] = np.random.poisson(8, size=(250, 21)) - np.random.randint(0, 4, size=(250, 21))

### [0.2] Features for df1 (independent variable of prediction model)

In [4]:
# Total
df1['Total'] = df1.loc[:, 'X1':'X14'].sum(axis=1)
# TS (X1~X14 are exisiting TS features))
for i in range(2, 15):
    col_name = f'TS{i}'
    start_col = f'X{i}'
    df1[col_name] = df1.loc[:, start_col:'X14'].sum(axis=1)
# CT
df1['tc'] = np.random.uniform(low=-14, high=15, size=500)  # creation time of the user cookie, maybe cleaned after exp start
df1['tf'] = np.random.uniform(low=15, high=28, size=500)  # time of the first entrance of the user in the experiment
df1['te'] = 15 # experiment begins at
df1['CT1'] = df1['te'] - df1['tc']
df1['CT2'] = df1['tf'] - df1['te']
df1['CT3'] = df1['tf'] - df1['tc']
# TrTS (to simplify we only use discrete Fourier transform to transform TS2 to TS14)
for i in range(2, 15):
    ts_col = f'TS{i}'
    tr_col = f'Tr{ts_col}'
    ts_data = df1[ts_col].values
    tr_data = np.abs(fft(ts_data)) / len(ts_data) # Normalized DFT
    df1[tr_col] = tr_data

### [0.3] Sample data

In [5]:
# Metrics
df1.iloc[0:3, 0:21]

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21
0,6,7,6,3,7,4,7,3,3,3,...,2,6,3,8,5,8,4,2,7,4
1,4,7,4,9,8,4,3,9,4,4,...,11,6,9,9,6,7,8,6,7,4
2,5,10,7,9,6,7,4,6,9,4,...,8,2,4,4,4,2,5,10,3,7


In [6]:
# Features
df1.iloc[0:3, 22:41]

,TS2,TS3,TS4,TS5,TS6,TS7,TS8,TS9,TS10,TS11,TS12,TS13,TS14,tc,tf,te,CT1,CT2,CT3
0,57.0,50.0,44.0,41.0,34.0,30.0,23.0,20.0,17.0,14.0,11.0,9.0,3.0,10.435050,23.802408,15,4.564950,8.802408,13.367358
1,82.0,75.0,71.0,62.0,54.0,50.0,47.0,38.0,34.0,30.0,26.0,15.0,9.0,6.472804,25.123187,15,8.527196,10.123187,18.650383
2,82.0,72.0,65.0,56.0,50.0,43.0,39.0,33.0,24.0,20.0,14.0,6.0,4.0,-9.325639,20.950871,15,24.325639,5.950871,30.276510


In [7]:
# Features
df1.iloc[0:3, 42:]

,TrTS3,TrTS4,TrTS5,TrTS6,TrTS7,TrTS8,TrTS9,TrTS10,TrTS11,TrTS12,TrTS13,TrTS14
0,78.274000,71.898000,65.318000,58.762000,52.400000,45.954000,39.106000,32.400000,25.838000,19.292000,12.744000,6.388000
1,0.143186,0.107638,0.153270,0.213820,0.328152,0.247500,0.166506,0.254937,0.216211,0.258194,0.191700,0.093245
2,0.448154,0.349335,0.312764,0.149984,0.161768,0.135436,0.025521,0.058356,0.101730,0.236970,0.131162,0.153359


### [0.4] Variance of metric

In [8]:
# variance of metirc is the target of our study
y_triangle = []
for i in range(15, 22):
    y_triangle = y_triangle + list(df1['X{}'.format(i)])
variance_baseline = np.var(y_triangle)
print("Baseline varaiance is : ", variance_baseline)

Baseline varaiance is :  7.4751362448979615


# [1]  Linear Regression Model & GBDT

### <span style="color:blue;">In this section we train 4 Linear Regression models and 4 Friedman’s gradient boosted decision trees, to control nRMSE, so as to decrease the variance of the metrics, and therefore furtherly raise the power of AB-test

## [1.1] Model Baseline: Linear Regression

In [9]:
from sklearn.linear_model import LinearRegression

### [1.1.1] Baseline1: linear prediction model  on feature "Total"(LR@Total)

In [10]:
# Only include 'Total'
X = df1['Total'].values.reshape(-1, 1)
MSE = 0
mean = 0
y_triangle = [] # the novel metric introduce in this paper, equals to X - X~
df_LR_Total = pd.DataFrame()
for i in range(15, 22):
    Y = df1['X{}'.format(i)].values.reshape(-1, 1)
    model = LinearRegression()
    model.fit(X, Y)
    Y_pred = model.predict(X)
    error = np.square(Y - Y_pred).sum()
    MSE = MSE + error
    mean = mean + Y.sum()
    y_triangle = y_triangle + list(Y-Y_pred)
    df_LR_Total[f'X{i-14}'] = Y_pred.reshape(-1)
RMSE_111 = np.sqrt(MSE/(500*7))
nRMSE_111 = RMSE_111/(mean/(500*7))
variance_111 = np.var(y_triangle)
print('Variance for baseline model 1: ', variance_111)
print('nRMSE for baseline model 1: ', nRMSE_111)

Variance for baseline model 1:  7.450328522490009
nRMSE for baseline model 1:  0.4197614778668067


### [1.1.2] Baseline2: linear prediction model  on feature "Total" and "TS"(LR@Total∪TS)

In [11]:
# X1~X14; Total; TS2~TS14 as independen variable
X = df1.iloc[:, list(range(1, 15)) + list(range(22, 35))]
MSE = 0
y_triangle = [] # the novel metric introduce in this paper, equals to X - X~
for i in range(15, 22):
    Y = df1['X{}'.format(i)].values.reshape(-1, 1)
    model = LinearRegression()
    model.fit(X, Y)
    Y_pred = model.predict(X)
    error = np.square(Y - Y_pred).sum()
    MSE = MSE + error
    mean = mean + Y.sum()
    y_triangle = y_triangle + list(Y-Y_pred)
RMSE_112 = np.sqrt(MSE/(500*7))
nRMSE_112 = RMSE_112/(mean/(500*7))
variance_112 = np.var(y_triangle)
print('Variance for baseline model 2: ', variance_112)
print('nRMSE for baseline model 2: ', nRMSE_112)

Variance for baseline model 2:  6.211796557738894
nRMSE for baseline model 2:  0.19164381913598585


### [1.1.3] Baseline3: linear prediction model on feature "Total" and "TS" and "CT"(LR@Total∪TS∪CT)

In [12]:
# X1~X14; Total; TS2~TS14, TrTS as independent variable
X = df1.iloc[:, list(range(1, 15)) + list(range(22, 35)) + list(range(41, 54))]
MSE = 0
y_triangle = [] # the novel metric introduce in this paper, equals to X - X~
for i in range(15, 22):
    Y = df1['X{}'.format(i)].values.reshape(-1, 1)
    model = LinearRegression()
    model.fit(X, Y)
    Y_pred = model.predict(X)
    error = np.square(Y - Y_pred).sum()
    MSE = MSE + error
    mean = mean + Y.sum()
    y_triangle = y_triangle + list(Y-Y_pred)
RMSE_113 = np.sqrt(MSE/(500*7))
nRMSE_113 = RMSE_113/(mean/(500*7))
variance_113= np.var(y_triangle)
print('Variance for baseline model 3: ', variance_113)
print('nRMSE for baseline model 3: ', nRMSE_113)

Variance for baseline model 3:  6.049716460986249
nRMSE for baseline model 3:  0.12608438864991559


### [1.1.4] Baseline4: linear prediction model  on feature "Total"(LR@All)

In [13]:
# X1~X14; Total; TS2~TS14, CT1~3; TrTS as independent variable
X = df1.iloc[:, list(range(1, 15)) + list(range(22, 35)) + list(range(38, 54))]
MSE = 0
y_triangle = [] # the novel metric introduce in this paper, equals to X - X~
for i in range(15, 22):
    Y = df1['X{}'.format(i)].values.reshape(-1, 1)
    model = LinearRegression()
    model.fit(X, Y)
    Y_pred = model.predict(X)
    error = np.square(Y - Y_pred).sum()
    MSE = MSE + error
    mean = mean + Y.sum()
    y_triangle = y_triangle + list(Y-Y_pred)
RMSE_114 = np.sqrt(MSE/(500*7))
nRMSE_114 = RMSE_114/(mean/(500*7))
variance_114 = np.var(y_triangle)
print('Variance for baseline model 4: ', variance_114)
print('nRMSE for baseline model 4: ', nRMSE_114)

Variance for baseline model 4:  6.041122419325466
nRMSE for baseline model 4:  0.0944964056742277


## [1.2]  Friedman’s gradient boosted decision trees

In [14]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import KBinsDiscretizer

### [1.2.1]  GBDT model on feature "Total" (DT@Total)

In [15]:
X = df1['Total'].values.reshape(-1, 1)
# The binning strategy for the features is with n_bins 4, instead of 64 in paper because our data is much less than original
binning = KBinsDiscretizer(n_bins=32, encode='ordinal', strategy='quantile')
X_bin = binning.fit_transform(X)
MSE = 0
mean = 0
y_triangle = [] # the novel metric introduce in this paper, equals to X - X~
for i in range(15, 22):
    Y = df1['X{}'.format(i)].values
    # loss function define as squared_error, which is MSE
    model = GradientBoostingRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, loss='squared_error')
    model.fit(X_bin, Y)
    Y_pred = model.predict(X_bin)
    error = np.square(Y - Y_pred).sum()
    MSE = MSE + error
    mean = mean + Y.sum()
    y_triangle = y_triangle + list(Y - Y_pred)
RMSE_121 = np.sqrt(MSE/(500*7))
nRMSE_121 = RMSE_121/(mean/(500*7))
variance_121 = np.var(y_triangle)
print('Variance for GBDT model 1: ', variance_121)
print('nRMSE for GBDT model 1: ', nRMSE_121)

C:\Users\Riho\anaconda3\envs\env_ds_310\lib\site-packages\sklearn\preprocessing\_discretization.py:291: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


Variance for GBDT model 1:  7.074251503770762
nRMSE for GBDT model 1:  0.40902995690294247


### [1.2.2] GBDT model on feature "Total" and "TS"(DT@Total∪TS)

In [16]:
# X1~X14; Total; TS2~TS14 as independen variable
X = df1.iloc[:, list(range(1, 15)) + list(range(22, 35))]
# The binning strategy for the features is with n_bins 4, instead of 64 in paper because our data is much less than original
binning = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')
X_bin = binning.fit_transform(X)
MSE = 0
mean = 0
y_triangle = [] # the novel metric introduce in this paper, equals to X - X~
for i in range(15, 22):
    Y = df1['X{}'.format(i)].values
    # loss function define as squared_error, which is MSE
    model = GradientBoostingRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, loss='squared_error')
    model.fit(X_bin, Y)
    Y_pred = model.predict(X_bin)
    error = np.square(Y - Y_pred).sum()
    MSE = MSE + error
    mean = mean + Y.sum()
    y_triangle = y_triangle + list(Y - Y_pred)
RMSE_122 = np.sqrt(MSE/(500*7))
nRMSE_122 = RMSE_122/(mean/(500*7))
variance_122 = np.var(y_triangle)
print('Variance for GBDT model 2: ', variance_122)
print('nRMSE for GBDT model 2: ', nRMSE_122)

Variance for GBDT model 2:  3.022212606086036
nRMSE for GBDT model 2:  0.26734826800441835


### [1.2.3] GBDT model on feature "Total" and "TS" and "CT"(DT@Total∪TS∪CT)

In [17]:
# X1~X14; Total; TS2~TS14, TrTS as independent variable
X = df1.iloc[:, list(range(1, 15)) + list(range(22, 35)) + list(range(41, 54))]
# The binning strategy for the features is with n_bins 4, instead of 64 in paper because our data is much less than original
binning = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')
X_bin = binning.fit_transform(X)
MSE = 0
mean = 0
y_triangle = [] # the novel metric introduce in this paper, equals to X - X~
for i in range(15, 22):
    Y = df1['X{}'.format(i)].values
    # loss function define as squared_error, which is MSE
    model = GradientBoostingRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, loss='squared_error')
    model.fit(X_bin, Y)
    Y_pred = model.predict(X_bin)
    error = np.square(Y - Y_pred).sum()
    MSE = MSE + error
    mean = mean + Y.sum()
    y_triangle = y_triangle + list(Y - Y_pred)
RMSE_123 = np.sqrt(MSE/(500*7))
nRMSE_123 = RMSE_123/(mean/(500*7))
variance_123 = np.var(y_triangle)
print('Variance for GBDT model 3: ', variance_123)
print('nRMSE for GBDT model 3: ', nRMSE_123)

Variance for GBDT model 3:  2.5367526405679004
nRMSE for GBDT model 3:  0.2449367376349107


### [1.2.4] GBDT model on feature "Total" and "TS" and "CT"(DT@Total∪TS∪CT)

In [18]:
# X1~X14; Total; TS2~TS14, CT1~3; TrTS as independent variable
X = df1.iloc[:, list(range(1, 15)) + list(range(22, 35)) + list(range(38, 54))]
# The binning strategy for the features is with n_bins 4, instead of 64 in paper because our data is much less than original
binning = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')
X_bin = binning.fit_transform(X)
MSE = 0
mean = 0
y_triangle = [] # the novel metric introduce in this paper, equals to X - X~
df_GBDT_ALL = pd.DataFrame()
for i in range(15, 22):
    Y = df1['X{}'.format(i)].values
    # loss function define as squared_error, which is MSE
    model = GradientBoostingRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, loss='squared_error')
    model.fit(X_bin, Y)
    Y_pred = model.predict(X_bin)
    error = np.square(Y - Y_pred).sum()
    MSE = MSE + error
    mean = mean + Y.sum()
    y_triangle = y_triangle + list(Y - Y_pred)
    df_GBDT_ALL[f'X{i-14}'] = Y_pred.reshape(-1)
RMSE_124 = np.sqrt(MSE/(500*7))
nRMSE_124 = RMSE_124/(mean/(500*7))
variance_124 = np.var(y_triangle)
print('Variance for GBDT model 4: ', variance_124)
print('nRMSE for GBDT model 4: ', nRMSE_124)

Variance for GBDT model 4:  2.432676123045691
nRMSE for GBDT model 4:  0.23985955012033208


# [3] Tables summarize the result

### Table 1: Comparison of feature sets and models in terms of the average value of nRMSE(relative improvement w.r.t. the first row)

In [19]:
df_table1 = pd.DataFrame(['Total', 'Total∪TS', 'Total∪TS∪CT', 'ALL']).rename(columns = {0:'Feature Set'})
df_table1['LR nRMSE'] = [nRMSE_111, nRMSE_112, nRMSE_113, nRMSE_114]
df_table1['LR nRMSE %'] = df_table1['LR nRMSE']/nRMSE_111
df_table1['LR nRMSE w.r.t previous row'] = df_table1['LR nRMSE %'] - ([1] + list(df_table1['LR nRMSE %'])[:-1])
df_table1['GBDT nRMSE'] = [nRMSE_121, nRMSE_122, nRMSE_123, nRMSE_124]
df_table1['GBDT nRMSE %'] = df_table1['GBDT nRMSE']/nRMSE_121
df_table1['GBDT nRMSE w.r.t previous row'] = df_table1['GBDT nRMSE %'] - ([1] + list(df_table1['GBDT nRMSE %'])[:-1])
df_table1[['LR nRMSE %', 'LR nRMSE w.r.t previous row', 'GBDT nRMSE %', 'GBDT nRMSE w.r.t previous row']] =\
    df_table1[['LR nRMSE %', 'LR nRMSE w.r.t previous row',  'GBDT nRMSE %', 'GBDT nRMSE w.r.t previous row']].applymap('{:.2%}'.format)
df_table1

,Feature Set,LR nRMSE,LR nRMSE %,LR nRMSE w.r.t previous row,GBDT nRMSE,GBDT nRMSE %,GBDT nRMSE w.r.t previous row
0,Total,0.419761,100.00%,0.00%,0.409030,100.00%,0.00%
1,Total∪TS,0.191644,45.66%,-54.34%,0.267348,65.36%,-34.64%
2,Total∪TS∪CT,0.126084,30.04%,-15.62%,0.244937,59.88%,-5.48%
3,ALL,0.094496,22.51%,-7.53%,0.239860,58.64%,-1.24%


### Table 2: Comparison of feature sets and models in terms of the average Variance Reduction rate(VRrate) (relative improvement w.r.t. the original data & previous row)

In [20]:
df_table2 = pd.DataFrame(['Baseline' ,'Total', 'Total∪TS', 'Total∪TS∪CT', 'ALL']).rename(columns = {0:'Feature Set'})
df_table2['LR variance'] = [variance_baseline, variance_111, variance_112, variance_113, variance_114]
df_table2['LR VR %'] = df_table2['LR variance']/variance_baseline
df_table2['LR VR w.r.t previous row'] = df_table2['LR VR %'] - ([1] + list(df_table2['LR VR %'])[:-1])
df_table2['GBDT variance'] = [variance_baseline, variance_121, variance_122, variance_123, variance_124]
df_table2['GBDT VR %'] = df_table2['GBDT variance']/variance_baseline
df_table2['GBDT VR w.r.t previous row'] = df_table2['GBDT VR %'] - ([1] + list(df_table2['GBDT VR %'])[:-1])
df_table2[['LR VR %', 'LR VR w.r.t previous row', 'GBDT VR %', 'GBDT VR w.r.t previous row']] =\
    df_table2[['LR VR %', 'LR VR w.r.t previous row', 'GBDT VR %', 'GBDT VR w.r.t previous row']].applymap('{:.2%}'.format)
df_table2

,Feature Set,LR variance,LR VR %,LR VR w.r.t previous row,GBDT variance,GBDT VR %,GBDT VR w.r.t previous row
0,Baseline,7.475136,100.00%,0.00%,7.475136,100.00%,0.00%
1,Total,7.450329,99.67%,-0.33%,7.074252,94.64%,-5.36%
2,Total∪TS,6.211797,83.10%,-16.57%,3.022213,40.43%,-54.21%
3,Total∪TS∪CT,6.049716,80.93%,-2.17%,2.536753,33.94%,-6.49%
4,ALL,6.041122,80.82%,-0.11%,2.432676,32.54%,-1.39%


# [4] AB-test improvement

### <span style="color:blue;">In this section, we choose 2 models: DT@ALL (best performance) and LR@Total to running an ABtest on them, and comparing its performance with the ABtest using the original dataset

In [21]:
from scipy.stats import ttest_ind

In [22]:
def run_ab_test(df, name):
    control = df.iloc[:250]
    treatment = df.iloc[250:]
    p_values = []
    # Calculate p-value for all 7 days combined
    control_all = control.values.reshape(-1)
    treatment_all = treatment.values.reshape(-1)
    t, p = ttest_ind(control_all, treatment_all)
    p_values.append(p)

    return pd.DataFrame(p_values, index = [name]).rename(columns = {0 : 'ABtest pvalue'})

### [4.1] ABtest outcome

In [23]:
df_baseline = df1.iloc[:, 14:21]
pd.concat([run_ab_test(df_baseline, 'original data'), run_ab_test(df_LR_Total, 'LR@Total'), run_ab_test(df_GBDT_ALL, 'GBDT@ALL')])

,ABtest pvalue
original data,0.743250
LR@Total,0.585206
GBDT@ALL,0.325750


In [ ]:
# successfully improve the sensitivity of ttest